### Prerequisites

In [ ]:
%pip install pyodbc pypyodbc pymssql pandas langchain openai python-dotenv tabulate

### Load environment variables

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.


### Get dataframe from SQL

In [ ]:
import os
import struct
import pyodbc
import pandas as pd

server = os.getenv('SQL_HOST')
database = os.getenv('SQL_DATABASE')
username = os.getenv('SQL_USERNAME')
password = os.getenv('SQL_PASSWORD')

connect_string = 'DRIVER={ODBC Driver 18 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password + ';Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=30;'


print(f"Connecting string: {connect_string}")

def HandleHierarchyId(v):
    return str(v)

cnxn = pyodbc.connect(connect_string)
cnxn.add_output_converter(-151, HandleHierarchyId)

cursor = cnxn.cursor()
query = "select * from [Person].[Address];"
df = pd.read_sql(query, cnxn)
print(df.head(5))

In [ ]:
# Import Azure OpenAI
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import HumanMessage

import openai
import os

In [ ]:
OPENAI_API_TYPE = "azure"
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
OPENAI_API_VERSION = "2022-12-01"

openai.log = 'debug'

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

# Simply test the model
print(llm([HumanMessage(content="Tell me a joke")]))

In [ ]:
from langchain.agents import create_pandas_dataframe_agent
agent = create_pandas_dataframe_agent(llm, df, verbose=True)


In [ ]:
agent.run("what is address line 1 for id 2?")

In [ ]:
agent.run("list the name of unique cities")

In [ ]:
agent.run("how many unique state provinces are there?")

---

### Real Database Connection

In [ ]:
from langchain import SQLDatabaseChain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

conn_str = "mssql+pymssql://" + username + ":" + password + "@localhost:1433/" + database
print("connect to " + conn_str)

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

db = SQLDatabase.from_uri(conn_str)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("How many employees are there in HumanResources?")

In [ ]:
db_chain.run("list the name of unique cities")

In [ ]:
db_chain.run("how many tables are there?")